# Using Reinforcement Learning to Play a Fishing Game
*By Keegan Kochis and Landon Zweigle*

# Introduction
Ever since we have gotten into programming we have had a desire to do reinforcement learning, and this term project seemed like a perfect excuse to finally do it. When it came to deciding what to do RL on, we decided to use a [JavaFX](https://openjfx.io/) game Landon made a few years ago; a very simple fishing minigame where the goal is to put a bobber over a fish for a certain amount of time. This game had everything we were looking for; an evironment which is rather simple, but not too simple; and an easy way to communicate with it.

Because the game was built in Java and our RL algorithm would be implemented in Python, we needed a way to communicate between Python and Java. For this we decided to use socket programming. Further, because of the random movement of the fish, we knew we had to use a neural network to approximate the Q-table.